In [ ]:
!pip install torch datasets transformers


In [2]:
import json
import os
import random
import torch
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Carregar o tokenizer e o modelo
tokenizer = T5Tokenizer.from_pretrained("unicamp-dl/ptt5-small-portuguese-vocab")
model = T5ForConditionalGeneration.from_pretrained("unicamp-dl/ptt5-small-portuguese-vocab")

In [5]:
# Verificar se a GPU está disponível e mover o modelo para a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [6]:
# Carregar o dataset a partir de um arquivo JSON
with open('text_to_text.json', 'r') as f:
    data = json.load(f)

In [7]:

# Preparar os dados
def prepare_data(data):
    input_texts = []
    target_texts = []

    for item in data:
        question = item['question']
        answer = item['answer']

        # Formatar a entrada e saída no formato desejado
        input_texts.append(f"question: {question}")
        target_texts.append(f"answer: {answer}")

    return input_texts, target_texts

In [8]:
input_texts, target_texts = prepare_data(data)

In [9]:
# Separar o dataset em treino e validação (80% treino, 20% validação)
train_size = int(0.8 * len(input_texts))
train_data = input_texts[:train_size], target_texts[:train_size]
val_data = input_texts[train_size:], target_texts[train_size:]

In [10]:
# Converter para o formato Dataset do Hugging Face
train_dataset = Dataset.from_dict({"input_texts": train_data[0], "target_texts": train_data[1]})
val_dataset = Dataset.from_dict({"input_texts": val_data[0], "target_texts": val_data[1]})

In [11]:
# Tokenizar os dados
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_texts'], max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(examples['target_texts'], max_length=512, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='text',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=True,  # Habilitar precisão mista
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [14]:
# Usar o Trainer do Hugging Face para treinar o modelo
trainer = Trainer(
    model=model,                     # Modelo a ser treinado
    args=training_args,              # Argumentos de treinamento
    train_dataset=train_dataset,     # Dataset de treino
    eval_dataset=val_dataset         # Dataset de validação
)

In [ ]:
# Iniciar o treinamento
trainer.train()

# Salvando o modelo no diretório especificado
trainer.save_model()

# Salvar o modelo treinado
model.save_pretrained(training_args.output_dir, safe_serialization=True)
tokenizer.save_pretrained(training_args.output_dir)

Usando o modelo

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

# Carregando o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/ptt5-large-portuguese-vocab")
model = AutoModelWithLMHead.from_pretrained("unicamp-dl/ptt5-large-portuguese-vocab")

# Texto de entrada
input_text = "A ciência e a tecnologia têm avançado rapidamente no Brasil. Recentemente,"

# Tokenizando o texto de entrada
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Gerando o texto com o modelo. Vou definir um tamanho menor para o max_length
# e garantir que apenas uma sequência seja retornada com num_return_sequences=1.
# Ajuste os parâmetros conforme necessário para a tarefa que você está realizando.
outputs = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

# Decodificando a saída para obter o texto gerado
generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Imprimindo o texto gerado
for text in generated_texts:
    print(text)
